In [1]:
import ml_collections
import torch
import random
import utils
from dpm_solver_pp import NoiseScheduleVP, DPM_Solver
from absl import logging
import einops
import libs.autoencoder
import libs.clip
from torchvision.utils import save_image, make_grid
import torchvision.transforms as standard_transforms
import numpy as np
import clip
from PIL import Image
from libs.uvit_t2i import UViT

attention mode is math


In [2]:
from run_train import stable_diffusion_beta_schedule, split_joint, combine_joint, Schedule

In [3]:
seed = 1234 #@param {type:"number"}
steps = 50 #@param {type:"slider", min:0, max:100, step:1}
cfg_scale = 8 #@param {type:"slider", min:0, max:10, step:0.1}
n_samples = 2 #@param {type:"number"}
nrow = 2 #@param {type:"number"}
data_type = 1
output_path = '/Users/jihyeonje/unidiffuser/test'
device = 'cpu'


In [4]:
nnet = UViT(
    img_size=1296,
    in_chans=3,
    patch_size=4,
    embed_dim=100,
    depth=12,
    num_heads=10,
    mlp_ratio=4,
    qkv_bias=False,
    mlp_time_embed=False,
    clip_dim=19,
    num_clip_token=77
)


nnet.to(device)
nnet.load_state_dict(torch.load('/Users/jihyeonje/unidiffuser/test/5.ckpt/nnet.pth', map_location='cpu'))
nnet.eval()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jihyeonje/unidiffuser/test/5.ckpt/nnet.pth'

In [ ]:
_betas = stable_diffusion_beta_schedule()
N = len(_betas)
_schedule = Schedule(_betas)


In [ ]:

def joint_nnet(x, timesteps):
    z, text = split_joint(x)
    z_out, text_out = nnet(z, t_prot=timesteps, t_lig=timesteps, context = text)
    if len(z_out.shape)==3:
        z_out = torch.unsqueeze(z_out, 0)
        text_out = torch.unsqueeze(text_out, 0)
    x_out = combine_joint(z_out, text_out)

    return x_out

def sample_fn(_n_samples, sample_steps):
    _z_init = torch.randn(_n_samples, *(3,1296,4), device=device)
    _t_init = torch.randn(_n_samples, *(100,19), device=device)
    _x_init = combine_joint(_z_init, _t_init)
    noise_schedule = NoiseScheduleVP(schedule='discrete', betas=torch.tensor(_betas, device=device).float())

    def model_fn(x, t_continuous):
        t = t_continuous * _schedule.N
        return joint_nnet(_x_init, t)

    dpm_solver = DPM_Solver(model_fn, noise_schedule, predict_x0=True, thresholding=False)
    _z = dpm_solver.sample(_x_init, steps=sample_steps, eps=1. / _schedule.N, T=1.)
    prot, lig = split_joint(_z)
    return prot, lig





In [ ]:
_z, _text = sample_fn(_n_samples=5, sample_steps=5)

In [ ]:
xyz = _text[3][:,:3]
one_hot = _text[3][:,3:]
one_hot = np.argmax(one_hot, axis=1)

In [ ]:
from datasets import atom_encoder, atom_decoder
from chem_utils import get_bond_order, allowed_bonds, draw_mol, BasicMolecularMetrics

In [5]:
draw_mol(xyz)

NameError: name 'draw_mol' is not defined